## Analys av Sambandet Mellan BNP per Capita och Förväntad Livslängd

## Inledning
Denna notebook utforskar sambandet mellan BNP per capita och förväntad livslängd över olika länder. Analysen är grundad på två datasets: ett för BNP per capita och ett annat för förväntad livslängd.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Datainläsning och Förberedelse

In [ ]:
# Ladda in datasets
life_expectancy_path = 'LifeExpectancyData.csv'
gdp_per_capita_path = 'gdp_per_capita.csv'

life_expectancy_data = pd.read_csv(life_expectancy_path)
gdp_per_capita = pd.read_csv(gdp_per_capita_path)

# Förbered data
life_expectancy_df = life_expectancy_data[['Country', 'Year', 'Life expectancy ']].rename(columns={'Life expectancy ': 'Life_expectancy'})
gdp_per_capita_df = gdp_per_capita.melt(id_vars=['Country Name'], var_name='Year', value_name='GDP_per_capita', value_vars=gdp_per_capita.columns[2:-1])
gdp_per_capita_df['Year'] = gdp_per_capita_df['Year'].astype(int)

# Sammanfoga datasets
merged_df = pd.merge(life_expectancy_df, gdp_per_capita_df, left_on=['Country', 'Year'], right_on=['Country Name', 'Year'], how='inner').drop('Country Name', axis=1)

## Data Cleaning

Under datarengöringsfasen behandlades saknade värden genom att fylla dem med medelvärdet. BNP per capita-datasetet omstrukturerades från brett till långt format för enkel analys över tid. Endast relevanta kolumner behölls för att fokusera på variabler av intresse för undersökningen.

In [ ]:


# Fyll saknade värden med medelvärdet för numeriska kolumner.
life_expectancy_data.fillna(life_expectancy_data.mean(numeric_only=True), inplace=True)
gdp_per_capita.fillna(gdp_per_capita.mean(numeric_only=True), inplace=True)

# Transformera BNP per capita-datasetet från bred till lång format
gdp_per_capita_long = gdp_per_capita.melt(id_vars=['Country Name', 'Code'], var_name='Year', value_name='GDP Per Capita')
gdp_per_capita_long['Year'] = pd.to_numeric(gdp_per_capita_long['Year'], errors='coerce')
gdp_per_capita_long.dropna(subset=['Year'], inplace=True)  
gdp_per_capita_long['Year'] = gdp_per_capita_long['Year'].astype(int)

# Slår ihop datasets på 'Country' och 'Year'
merged_data = pd.merge(life_expectancy_data, gdp_per_capita_long, left_on=['Country', 'Year'], right_on=['Country Name', 'Year'], how='inner')
columns_to_keep = ['Country', 'Year', 'Life expectancy ', 'GDP', 'GDP Per Capita', 'Status']

# Filtrera det sammanslagna datasetet för att behålla endast de relevanta kolumnerna
cleaned_data = merged_data[columns_to_keep]

# Spara cleaned dataset
cleaned_data.to_csv('/Users/p/Desktop/AI_Dev/Python forts/Pandas/cleaned_data.csv', index=False)


## Visualisering av sambandet mellan BNP per Capita och Förväntad Livslängd


In [ ]:

correlation = cleaned_data[['Life expectancy ', 'GDP Per Capita']].corr()

# Visualisering
sns.set_theme(style="whitegrid")
plt.figure(figsize=(10, 6))

# Scatter plot för att visa sambandet mellan BNP per capita och förväntad livslängd
sns.scatterplot(data=cleaned_data, x='GDP Per Capita', y='Life expectancy ', alpha=0.6)
plt.title('Sambandet mellan BNP per Capita och Förväntad Livslängd')
plt.xlabel('BNP per Capita (USD)')
plt.ylabel('Förväntad Livslängd (År)')

# Använder en logaritmisk skala för x-axeln för att bättre visa datan
plt.xscale('log')
plt.grid(True, which="both", ls="--")
plt.show()

#correlation matrix
correlation


Denna kod genererar en scatter plot som visar sambandet mellan BNP per capita och förväntad livslängd för olika länder. Scatter plotten använder en logaritmisk skala för x-axeln för att bättre visa datan och inkluderar en grid för ökad tydlighet. Dessutom beräknas korrelationskoefficienten för att kvantifiera styrkan i sambandet mellan de två variablerna.

# Dataanalys

In [ ]:
# Korrigera kolumnnamnen
correlation = cleaned_data[['Life expectancy ', 'GDP Per Capita']].corr()

# Visualisering
sns.set_theme(style="whitegrid")
plt.figure(figsize=(10, 6))
sns.scatterplot(data=cleaned_data, x='GDP Per Capita', y='Life expectancy ', alpha=0.6)
plt.title('Sambandet mellan BNP per Capita och Förväntad Livslängd')
plt.xlabel('BNP per Capita (USD)')
plt.ylabel('Förväntad Livslängd (År)')
plt.xscale('log')
plt.grid(True, which="both", ls="--")
plt.show()

# Visa korrelationsmatrisen
correlation


## Diskussion
Analysen visar på ett positivt samband mellan BNP per capita och förväntad livslängd, vilket indikerar att högre ekonomiskt välstånd kan associeras med bättre hälsa och längre livslängd. Detta samband är dock inte perfekt och det finns variationer som kan bero på andra faktorer som hälsopolitik, tillgång till vård och livsstil.

## Analys av Förväntad Livslängd och dess Samband med BNP per Capita


I detta avsnitt utforskar vi sambandet mellan förväntad livslängd och BNP per capita genom att analysera data från olika länder. Analysen inkluderar beräkning av korrelation för att identifiera styrkan och riktningen på sambandet mellan dessa två variabler. Genom att använda visualiseringstekniker som scatterplots har vi belyst hur ekonomisk välstånd, mätt genom BNP per capita, påverkar folkhälsan, särskilt den förväntade livslängden. Vidare identifierar vi länder vars förväntade livslängd avviker positivt från genomsnittet med mer än en standardavvikelse, vilket ger insikter i hur vissa nationer överträffar andra trots liknande ekonomiska förutsättningar.

In [ ]:

# Ladda in den rensade datan
cleaned_data = pd.read_csv('/Users/p/Desktop/AI_Dev/Python forts/Pandas/cleaned_data.csv')

# Fortsätt med beräkningarna
mean_life_expectancy = cleaned_data['Life expectancy '].mean()
std_life_expectancy = cleaned_data['Life expectancy '].std()

# Identifiera länder med livslängd en standardavvikelse över medelvärdet
countries_above_std = cleaned_data[cleaned_data['Life expectancy '] > mean_life_expectancy + std_life_expectancy]

# Beräkna medelvärde och standardavvikelse för BNP per Capita för vidare analys
mean_gdp_per_capita = cleaned_data['GDP Per Capita'].mean()
std_gdp_per_capita = cleaned_data['GDP Per Capita'].std()

# Visa grundläggande statistik och de första raderna i countries_above_std för granskning
print(mean_life_expectancy, std_life_expectancy)
print(countries_above_std.head())
print(mean_gdp_per_capita, std_gdp_per_capita)


Vår analys visar att länder med högre BNP per capita, såsom Australien, tenderar att ha en högre förväntad livslängd, vilket understryker sambandet mellan ekonomisk utveckling och folkhälsa. Detta framhäver vikten av socioekonomiska investeringar för att förbättra livslängden och minska globala hälsoklyftor.

## Ekonomisk Rikedom kontra Förväntad Livslängd


I denna analys utforskar vi om högre BNP och BNP per capita direkt korrelerar med längre förväntad livslängd. Vi granskar länder med hög förväntad livslängd men relativt låg ekonomisk output för att identifiera om andra faktorer än ekonomisk rikedom bidrar till folkhälsan.



In [ ]:
# Identifiera länder med förväntad livslängd över medel men BNP per capita under medel
higher_life_expectancy = cleaned_data[cleaned_data['Life expectancy '] > cleaned_data['Life expectancy '].mean()]
lower_gdp = higher_life_expectancy[higher_life_expectancy['GDP Per Capita'] < cleaned_data['GDP Per Capita'].mean()]

# Visualisera resultaten

sns.scatterplot(data=lower_gdp, x='GDP Per Capita', y='Life expectancy ', hue='Status')
plt.title('Hög Förväntad Livslängd trots Låg BNP per Capita')
plt.xlabel('BNP per Capita (USD)')
plt.ylabel('Förväntad Livslängd (År)')
plt.show()

# Skriv ut länder som uppfyller kriterierna
print(lower_gdp[['Country', 'Year', 'Life expectancy ', 'GDP Per Capita']])


Vår analys avslöjar att länder som Albanien och Vanuatu uppnår en förvånansvärt hög förväntad livslängd trots att de har en BNP per capita under genomsnittet, vilket pekar på att ekonomisk rikedom inte är den enda faktorn för god folkhälsa. Detta understryker betydelsen av andra faktorer som utbildning, tillgång till hälsovård och samhällsstöd, vilket kan ha en stark påverkan på livslängden. Dessa insikter belyser vikten av att utveckla hälsofrämjande strategier som tar hänsyn till en bredare uppsättning sociala och ekonomiska förhållanden.